<a href="https://colab.research.google.com/github/josemoragonzalez/ejemplosColab/blob/master/ApiChC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tercer Ejercicio, Api desarrolladores ChC

In [1]:
import pandas as pd
import requests

In [2]:
# Agregar tu ticket
ticket = 'C495FBFD-6480-46C6-8A1A-EAE762EC0271'

In [3]:
def compradores(ticket):
  url = 'http://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarComprador?ticket='+ticket
  r = requests.get(url)
  comp_json = r.json()
  df = pd.json_normalize(comp_json['listaEmpresas']) 
  return df

In [4]:
dfcomp = compradores(ticket)
dfcomp

,CodigoEmpresa,NombreEmpresa
0,1034640,CORPORACION MUNICIPAL DE PEÑALOLEN PARA EL DE...
1,7311,Hospital de enfermedades InfecciDr. Lucio Cór...
2,7086,AGCID
3,1224636,Agencia Chilena de Eficiencia Energética
4,7193,Agencia de Promoción de la Inversión Extranjera
...,...,...
856,96996,Universidad de Valparaíso
857,99805,Universidad del Bío-Bío
858,164268,UNIVERSIDAD METROPOLITANA DE CIENCIAS DE LA ED...
859,126109,Universidad Tecnológica Metropolitana


In [5]:
def ocxfecha(fecha,ticket):
  url = 'http://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?fecha='+fecha+'&ticket='+ticket
  r = requests.get(url)
  comp_json = r.json()
  df = pd.json_normalize(comp_json['Listado']) 
  return df['Codigo']

In [19]:
dfocs = ocxfecha('01012018', ticket)
len(dfocs)

195

In [20]:
def ocd(codigo, ticket):
  import time
  time.sleep(2)
  url = 'http://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?codigo='+codigo+'&ticket='+ticket
  r = requests.get(url)
  comp_json = r.json()
  df = pd.json_normalize(comp_json['Listado']) 
  dfred = df[['Codigo', 'Estado', 'TipoMoneda', 'Total', 'Fechas.FechaEnvio', 'Comprador.NombreOrganismo', 'Proveedor.Nombre']]
  return dfred

In [21]:
dftest = ocd('1057501-6535-SE20', ticket)
dftest

,Codigo,Estado,TipoMoneda,Total,Fechas.FechaEnvio,Comprador.NombreOrganismo,Proveedor.Nombre
0,1057501-6535-SE20,Aceptada,CLP,10017000.0,2020-07-15T17:57:17.413,Complejo Asistencial Dr. Sotero del Rio,CICARSOT S. A.


In [13]:
#pd.json_normalize(dftest['Items.Listado'].to_dict()[0])#

In [38]:
def oc_fecha_con_detalle(fecha, ticket):
  listoc = ocxfecha(fecha,ticket)
  dfoctotal = pd.DataFrame()
  for oc in listoc:
    dfoctotal = (dfoctotal.append(ocd(oc, ticket)))

  return dfoctotal

In [39]:
import time
dfprueba =  oc_fecha_con_detalle('01012020', ticket)
dfprueba

,Codigo,Estado,TipoMoneda,Total,Fechas.FechaEnvio,Comprador.NombreOrganismo,Proveedor.Nombre
0,1057384-402-SE19,Aceptada,CLP,250000.0,2019-12-31T11:32:46.743,Hospital Tocopilla,das ohr
0,1057402-10483-SE19,En proceso,CLP,900000.0,2019-12-12T10:37:23.943,Hospital Dr. Lautaro Navarro Avaria,JOSE LUIS
0,1057402-10756-SE19,Aceptada,CLP,1316467.0,2019-12-31T13:18:35.683,Hospital Dr. Lautaro Navarro Avaria,Carola Andrea
0,1057402-10915-CM19,Aceptada,CLP,2082690.0,2019-12-30T17:25:04.627,Hospital Dr. Lautaro Navarro Avaria,Johnson & Johnson Medical
0,1057402-10997-SE19,Aceptada,CLP,360570.0,2019-12-30T18:19:51.637,Hospital Dr. Lautaro Navarro Avaria,Johnson & Johnson Medical
...,...,...,...,...,...,...,...
0,762514-35-SE19,Aceptada,CLP,710144.0,2019-04-05T15:07:00.26,CAPREDENA,NOVOFARMA SERVICE S A
0,812261-1758-SE19,Aceptada,CLP,5486000.0,2019-12-27T11:51:12.757,HOSPITAL CLINICO METROPOLITANO LA FLORIDA DRA ...,FastCare SpA
0,879-345-CM19,Aceptada,CLP,128029.0,2019-12-31T21:14:31.933,Instituto Nacional de la Juventud - MIDEPLAN,Sky Airline
0,924-965-SE19,Aceptada,CLP,1957550.0,2019-12-28T11:01:21.193,Subsecretaría de Salud Pública,Climalva Climatizacion Limitada


In [24]:
dfprueba

""
